<a href="https://colab.research.google.com/github/cunhamaicon/catsxdogs/blob/master/catsxdogs_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning - Problema cats x dogs

Implementação de rede convolucional usando transfer learning para diferenciação das categorias gato e cachorro.

 O banco de dados original se encontra [aqui](https://www.kaggle.com/c/dogs-vs-cats). Dentre essas imagens foram separadas 8000 imagens para treinamento e 2000 imagens para teste.

Todo o código foi feito no Colab e pode ser compilado online em [https://colab.research.google.com](https://colab.research.google.com).


** Autor: Maicon Henrique Cunha**

Github: [https://github.com/cunhamaicon](https://github.com/cunhamaicon)

## Procedimentos Iniciais

Apagar a pasta catsxdogs caso algum novo dado seja incluído na pasta:

In [0]:
#!rm -rf catsxdogs

Download da pasta:

In [0]:
!git clone https://github.com/cunhamaicon/catsxdogs

Importação dos pacotes:

In [3]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout

Using TensorFlow backend.


## CNN - Transfer learning:

Importando o modelo MobileNet que foi previamente treinado no ImageNet e descartando a última camada de neurônios:

In [0]:
model=MobileNet(weights='imagenet',include_top=False)

Criando a saída do modelo MobileNet:

In [0]:
x=model.output
x=GlobalAveragePooling2D()(x)

Adicionando uma camada intermediária e a camada final:

In [0]:
x=Dense(50,activation='relu')(x)
preds=Dense(1,activation='sigmoid')(x) 
model=Model(inputs=model.input,outputs=preds)

Visualizando todas as camadas da nova rede criada usando o modelo MobileNetV2:

In [7]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

Definindo qual camada da rede será treinada. Nesse caso somente as duas últimas camadas adicionadas:

In [0]:
for layer in model.layers[:88]:
    layer.trainable=False
for layer in model.layers[88:]:
    layer.trainable=True

## ImageDataGenerator

Definindo o tamanho de cada batch:

In [0]:
batch_size = 32

Cada imagem do banco será apresentada a rede de uma forma diferente através do ImageDataGenerator:

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   height_shift_range=0.3,
                                   width_shift_range=0.3,
                                   rotation_range=50,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('catsxdogs/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('catsxdogs/test_set',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Treinamento

Definindo os parâmetros de compilação da rede:

In [0]:
model.compile(optimizer=Adam(lr = 0.0001),loss='binary_crossentropy',metrics=['accuracy'])

Fazendo o treinamento da rede:

In [12]:
history = model.fit_generator(generator=training_set,
                   steps_per_epoch=8000/batch_size,
                   epochs=10,
                   validation_data = test_set,
                   validation_steps = 2000/batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
250/250 [==============================] - 107s 430ms/step - loss: 0.3477 - acc: 0.8405 - val_loss: 0.1113 - val_acc: 0.9570
Epoch 2/10
250/250 [==============================] - 102s 410ms/step - loss: 0.2045 - acc: 0.9169 - val_loss: 0.0796 - val_acc: 0.9685
Epoch 3/10
250/250 [==============================] - 100s 399ms/step - loss: 0.1858 - acc: 0.9220 - val_loss: 0.0873 - val_acc: 0.9630
Epoch 4/10
250/250 [==============================] - 100s 400ms/step - loss: 0.1789 - acc: 0.9254 - val_loss: 0.0640 - val_acc: 0.9750
Epoch 5/10
250/250 [==============================] - 104s 414ms/step - loss: 0.1645 - acc: 0.9290 - val_loss: 0.0708 - val_acc: 0.9715
Epoch 6/10
250/250 [==============================] - 100s 400ms/step - loss: 0.1631 - acc: 0.9311 - val_loss: 0.0716 - val_acc: 0.9720
Epoch 7/10
250/250 [==============================] - 100s 400ms/step - loss: 0.1593 - acc: 0.9361 - val_loss: 0.0715 - val_acc: 0.9710


Salvando o modelo para utilização futura:

In [0]:
model.save('catsxdogs_mobilenet.h5')
from google.colab import files
files.download('catsxdogs_mobilenet.h5')

## Previsão

Mostrando os arquivos da pasta single_prediction com imagens inéditas para a rede classificar:

In [15]:
ls catsxdogs/single_prediction

cat_or_dog_1.jpg  chino1.jpg  floyd2.jpg  floyd4.jpg
cat_or_dog_2.jpg  floyd1.jpg  floyd3.jpg


Escolhendo uma imagem da pasta single_prediction para fazer a previsão:

In [0]:
test_image = image.load_img('catsxdogs/single_prediction/chino1.jpg', target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

In [18]:
result = model.predict(test_image)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

cat
